![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib

import os
os.chdir('..')

import werkzeug
#from flask.scaffold import _endpoint_from_view_func
from werkzeug.utils import cached_property

import flask

#flask.helpers._endpoint_from_view_func = _endpoint_from_view_func

werkzeug.cached_property = cached_property

from flask import Flask
from flask_restplus import Api

In [3]:
# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/phishing.csv')
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


## Codificar variables categóricas
Relizar preprocesamiento de texto (URLs) para crear variables predictoras:

In [4]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
keywords = ['https', 'login', '.php', '.html', '@', 'sign']
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

# Definición de la variable largo de la URL
data['lenght'] = data.url.str.len() - 2

# Definición de la variable largo del dominio de la URL
domain = data.url.str.split('/', expand=True).iloc[:, 2]
data['lenght_domain'] = domain.str.len()

# Definición de la variable binaria que indica si es IP
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

# Definicón de la variable cuenta de 'com' en la URL
data['count_com'] = data.url.str.count('com')

data.head()

,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,http://www.subalipack.com/contact/images/sampl...,1,0,0,0,0,0,0,47,18,0,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1,0,0,0,0,0,0,73,41,0,0
2,http://theotheragency.com/confirmer/confirmer-...,1,0,0,0,0,0,0,92,18,0,1
3,http://aaalandscaping.com/components/com_smart...,1,0,0,0,0,0,0,172,18,0,3
4,http://paypal.com.confirm-key-21107316126168.s...,1,0,0,0,0,0,0,90,50,0,1


In [5]:
# Separación de variables predictoras (X) y variable de interes (y)
X = data.drop(['url', 'phishing'], axis=1)
y = data.phishing

## Entrenar y guardar el modelo

In [6]:
# Definición de modelo de clasificación Random Forest
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=3)
cross_val_score(clf, X, y, cv=10)

array([0.75825, 0.76025, 0.74625, 0.76225, 0.74875, 0.7675 , 0.7555 ,
       0.757  , 0.75975, 0.771  ])

In [7]:
# Entrenamiento del modelo de clasificación Random Forest
clf.fit(X, y)

RandomForestClassifier(max_depth=3, n_jobs=-1)

In [8]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'phishing_clf.pkl', compress=3)

['phishing_clf.pkl']

In [9]:
# Importar modelo y predicción
from model_deployment.m09_model_deployment import predict_proba

# Predicción de probabilidad de que un link sea phishing
predict_proba('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

0.7568911356308352

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [10]:
# Importación librerías
from flask import Flask
from flask_restplus import Api, Resource, fields

In [11]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict', 
     description='Phishing Classifier')

# Definición argumentos o parámetros de la API
parser = api.parser()
parser.add_argument(
    'URL', 
    type=str, 
    required=True, 
    help='URL to be analyzed', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [12]:
# Definición de la clase para disponibilización
@ns.route('/')
class PhishingApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict_proba(args['URL'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swaggerui/favicon-16x16.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 20:42:29] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 20:42:42] "GET /predict/?URL=www.amazon.com HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/yovany/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  F

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.